## Setup Drive and Data Folder

In [1]:
import os
os.chdir('..')

In [2]:
import pandas as pd
import os

STEPS_PER_BATCH=32
STEPS_PER_BATCH=32
PRECISION = 4 # 3 of digits to keep after the decimal point
SEQUENCE_LENGHT=6
RUNNING_ON_COLAB = False # we assume running on CoLab! Change to False if running locally.

*italicized text*

In [3]:
## mount gdrive

# !conda install pandas --yes
# from google.colab import drive
# drive.mount("/content/gdrive")

In [4]:
# If running locally, define current working path
path = os.getcwd()

# If Google colab
if RUNNING_ON_COLAB:
    path = "/content/gdrive/My Drive/Colab Notebooks/RandomForest_FeatureData"

print(path)

# define current data path. This is after we did classification. We have done some
# cleaning already.
data_path = path + '/fl_testbed/version2/data/initial'
print(data_path)

/home/jose/FL_AM_Defect-Detection
/home/jose/FL_AM_Defect-Detection/fl_testbed/version2/data/initial


## Util - add RUL column

In [85]:
csv_file = data_path + '/combined_offset_misalignment.csv'
df_temp = pd.read_csv(csv_file, chunksize=50000) 
df = pd.concat(df_temp, ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: '/home/fl_testbed/version2/data/initial/combined_offset_misalignment.csv'

In [ ]:
# Let's find the youngest & oldest timestamp

df['wf_start_time'] = pd.to_datetime(df['wf_start_time']) # make sure it is datetime

youngest = min(df.wf_start_time)
oldest = max(df.wf_start_time)
print(youngest)
print(oldest)
span = oldest - youngest
print(span)
print(span.total_seconds())

## Using Oldest - current to determine the RUL
df['rul'] = df['wf_start_time'].apply(lambda x: (oldest - x).total_seconds())

In [ ]:
print(path)

In [78]:
# save back with RUL
df.to_csv(path + '/fl_testbed/version2/data/initial/combined_offset_misalignment_with_RUL.csv')


OSError: Cannot save file into a non-existent directory: '/home/fl_testbed/version2/data/initial'

In [ ]:
!pwd && cd "fl_testbed/version2/data/initial"  && ls 

## LSTM Regression

In [5]:
## Common imports
colab = False

import pandas as pd
import numpy as np
import csv
import os

from sklearn import tree
from sklearn import metrics
from sklearn.metrics import f1_score, cohen_kappa_score

from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np

from sklearn.metrics import r2_score, mean_squared_error
# from pandas_ml import ConfusionMatrix

import matplotlib
import matplotlib.pyplot as plt
plt.style.available
%matplotlib inline

# to make this notebook's output stable across runs
RNDSEED = np.random.seed(39)

2023-05-03 23:30:47.982419: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 23:30:48.627212: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Exploring the data a bit & prep the df

In [6]:
## future, let's read the file
if colab == False:
    csv_file = data_path+"/combined_offset_misalignment_with_RUL.csv"
else:
    csv_file = path + '/results/combined_offset_misalignment_with_RUL.csv'
df_temp = pd.read_csv(csv_file, chunksize=50000) 
big_df = pd.concat(df_temp)

In [7]:
df = big_df # reset 

# drop unwanted cols
df = df[df.columns.drop(list(df.filter(regex='Unnamed')))] # drop Unnamed
#df = df[df.columns.drop(list(df.filter(regex='wf_start_time')))] # drop time column
# df = df[df.columns.drop(list(df.filter(regex='status')))] # drop status column



In [8]:
df.shape

(238722, 49)

In [9]:
df.columns

Index(['wf_start_time', 'Probe_1_AxialDirection_CrestFactor_g~g',
       'Probe_1_AxialDirection_DerivedPeak_g',
       'Probe_1_AxialDirection_Peak~Peak_g', 'Probe_1_AxialDirection_RMS_g',
       'Probe_1_AxialDirection_TruePeak_g',
       'Probe_1_AxialDirection_HighFrequency_grms',
       'Probe_1_AxialDirection_Kurtosis_g~g',
       'Probe_2_RadialVertical_CrestFactor_g~g',
       'Probe_2_RadialVertical_DerivedPeak_g',
       'Probe_2_RadialVertical_Peak~Peak_g', 'Probe_2_RadialVertical_RMS_g',
       'Probe_2_RadialVertical_TruePeak_g',
       'Probe_2_RadialVertical_HighFrequency_grms',
       'Probe_2_RadialVertical_Kurtosis_g~g',
       'Probe_3_RadialHorizontal_CrestFactor_g~g',
       'Probe_3_RadialHorizontal_DerivedPeak_g',
       'Probe_3_RadialHorizontal_Peak~Peak_g',
       'Probe_3_RadialHorizontal_RMS_g', 'Probe_3_RadialHorizontal_TruePeak_g',
       'Probe_3_RadialHorizontal_HighFrequency_grms',
       'Probe_3_RadialHorizontal_Kurtosis_g~g',
       'Probe_4_RadialVe

In [10]:
df

,wf_start_time,Probe_1_AxialDirection_CrestFactor_g~g,Probe_1_AxialDirection_DerivedPeak_g,Probe_1_AxialDirection_Peak~Peak_g,Probe_1_AxialDirection_RMS_g,Probe_1_AxialDirection_TruePeak_g,Probe_1_AxialDirection_HighFrequency_grms,Probe_1_AxialDirection_Kurtosis_g~g,Probe_2_RadialVertical_CrestFactor_g~g,Probe_2_RadialVertical_DerivedPeak_g,...,Probe_6_BearingRadial_RMS_g,Probe_6_BearingRadial_TruePeak_g,Probe_6_BearingRadial_HighFrequency_grms,Probe_6_BearingRadial_Kurtosis_g~g,Thermocouple 1_Value,Thermocouple 2_Value,Thermocouple 3_Value,Thermocouple 4_Value,status,rul
0,2022-08-30 19:17:41.000007,2.9679,0.1485,0.6232,0.1050,0.3116,0.0303,2.3544,3.0998,0.4020,...,0.0166,0.0687,0.0074,3.1289,72.5799,72.4725,72.5867,73.0246,6.25mm_Offset,176049.0
1,2022-08-30 19:17:42.000007,3.0104,0.1511,0.6261,0.1069,0.3217,0.0310,2.3787,3.0161,0.4009,...,0.0162,0.0618,0.0076,2.9855,72.5552,72.4137,72.5893,73.0349,6.25mm_Offset,176048.0
2,2022-08-30 19:17:43.000007,3.2165,0.1611,0.6889,0.1139,0.3664,0.0309,2.3040,3.2447,0.4001,...,0.0160,0.0595,0.0074,3.2605,72.5661,72.4391,72.5610,73.0723,6.25mm_Offset,176047.0
3,2022-08-30 19:17:44.000007,3.1404,0.1388,0.6156,0.0981,0.3081,0.0309,2.4416,3.1142,0.4004,...,0.0163,0.0867,0.0075,3.7580,72.5788,72.4654,72.5600,73.0508,6.25mm_Offset,176046.0
4,2022-08-30 19:17:45.000007,3.4300,0.1520,0.6706,0.1075,0.3686,0.0302,2.3783,3.1580,0.4001,...,0.0160,0.0680,0.0074,3.1019,72.5864,72.4441,72.5711,73.0678,6.25mm_Offset,176045.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238717,2022-08-15 21:15:56.000007,3.2750,0.1771,0.7867,0.1252,0.4102,0.0499,2.4770,3.2238,0.3979,...,0.0129,0.0545,0.0044,3.1400,73.5705,73.2741,73.3993,73.7809,1mm_Offset,1464954.0
238718,2022-08-15 21:15:57.000007,3.0359,0.1750,0.7433,0.1237,0.3756,0.0490,2.3456,3.0837,0.3984,...,0.0132,0.0555,0.0045,3.1751,73.6015,73.3187,73.4508,73.7080,1mm_Offset,1464953.0
238719,2022-08-15 21:15:58.000007,2.8791,0.1622,0.6565,0.1147,0.3302,0.0495,2.4121,3.2785,0.3998,...,0.0126,0.0490,0.0044,3.0785,73.6119,73.2891,73.4424,73.7252,1mm_Offset,1464952.0
238720,2022-08-15 21:15:59.000007,3.8311,0.1496,0.7588,0.1058,0.4053,0.0258,3.0219,5.0555,0.2655,...,0.0122,0.0558,0.0030,3.3732,73.6241,73.3149,73.4350,73.7799,1mm_Offset,1464951.0


In [11]:
df[df['status']=='6.25mm_Offset']['wf_start_time']

0        2022-08-30 19:17:41.000007
1        2022-08-30 19:17:42.000007
2        2022-08-30 19:17:43.000007
3        2022-08-30 19:17:44.000007
4        2022-08-30 19:17:45.000007
                    ...            
29583    2022-08-31 20:42:03.000007
29584    2022-08-31 20:42:05.000007
29585    2022-08-31 20:42:06.000007
29586    2022-08-31 20:42:07.000007
29587    2022-08-31 20:42:09.000007
Name: wf_start_time, Length: 29588, dtype: object

In [12]:
df.status.value_counts()

No_Offset        79205
1mm_Offset       73082
3mm_Offset       36248
6.25mm_Offset    29588
12.7mm_Offset    20599
Name: status, dtype: int64

In [ ]:
df=df[['rul','status','Probe_5_RadialHorizontal_DerivedPeak_g', 'Probe_2_RadialVertical_Kurtosis_g~g', 'Probe_1_AxialDirection_HighFrequency_grms', 'Probe_1_AxialDirection_DerivedPeak_g', 'Probe_6_BearingRadial_HighFrequency_grms', 'Probe_5_RadialHorizontal_TruePeak_g', 'Probe_3_RadialHorizontal_Peak~Peak_g', 'Probe_4_RadialVertical_HighFrequency_grms']]

### DataSplit

In [13]:
## true orignal ones
# data split
X_train, X_test, y_train, y_test = train_test_split(df.drop('rul', axis=1), df['rul'],test_size=0.20)



In [14]:
print(X_train.shape, y_train.shape)

(179041, 48) (179041,)


In [15]:
#VALIDATION:

# X_vals, X_test, y_vals, y_test = train_test_split(X_residual, y_residual,shuffle=False)


In [16]:
# print(X_vals.shape, y_vals.shape)
# print(X_test.shape, y_test.shape)

## Vals (FOR HYPERPARAMETER TUNNING)

In [17]:

# X_vals = X_vals[X_vals.columns.drop(list(X_vals.filter(regex='wf_start_time')))] # drop time column
# X_vals = X_vals[X_vals.columns.drop(list(X_vals.filter(regex='status')))] # drop status column

# out_seq = np.array(y_vals)#out_seq.reshape((len(out_seq), 1))
# # horizontally stack columns
# dataset = np.array(X_vals)#hstack((in_seq1, in_seq2))

# pd.DataFrame(dataset)




# # shift the target sample by one step
# out_seq = np.insert(out_seq, 0, 0)
# #pre

# out_seq = np.delete(out_seq, -1)
# # define generator
# n_input = SEQUENCE_LENGHT #leght of the sequence timesteps

# h_validation_generator = TimeseriesGenerator(dataset, out_seq, length=n_input, batch_size=STEPS_PER_BATCH)
# # print each sample
# # for i in range(len(validation_generator)):
# #     x, y = generator[i]
# #     print('%s => %s' % (x, y))



# Testing

In [18]:
X_test = X_test[X_test.columns.drop(list(X_test.filter(regex='wf_start_time')))] # drop time column
X_test = X_test[X_test.columns.drop(list(X_test.filter(regex='status')))] # drop status column


out_seq = np.array(y_test)#out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.array(X_test)#hstack((in_seq1, in_seq2))

pd.DataFrame(dataset)




# shift the target sample by one step
out_seq = np.insert(out_seq, 0, 0)
#pre

out_seq = np.delete(out_seq, -1)
# define generator
n_input = SEQUENCE_LENGHT #leght of the sequence timesteps
testing_generator = TimeseriesGenerator(dataset, out_seq, length=n_input, batch_size=STEPS_PER_BATCH)
# print each sample
# for i in range(len(testing_generator)):
#     x, y = generator[i]
#     print('%s => %s' % (x, y))



# Data observation

In [19]:
for i in range(len(testing_generator)):
    x, y = testing_generator[i]
    print(x.shape, y.shape)

(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46) (32,)
(32, 6, 46

In [20]:

print(len(testing_generator[0][0][0][0]))
print(testing_generator[0][0][0][0])
print(len(testing_generator[0][0][0]))
print(testing_generator[0][0][0])
print(len(testing_generator[0][0]))
print(testing_generator[0][0])
print(len(testing_generator[0]))
print(testing_generator[0])



46
[3.32970e+00 1.41700e-01 6.41900e-01 1.00200e-01 3.33600e-01 4.68000e-02
 2.78340e+00 3.01860e+00 4.21200e-01 1.73810e+00 2.97800e-01 8.99000e-01
 2.95000e-01 2.60680e+00 2.48500e+00 6.97700e-01 2.36980e+00 4.93400e-01
 1.22600e+00 4.92400e-01 2.29860e+00 3.48270e+00 2.39300e-01 1.17100e+00
 1.69200e-01 5.89300e-01 1.08500e-01 2.94480e+00 2.48490e+00 4.77200e-01
 1.64680e+00 3.37400e-01 8.38500e-01 3.34900e-01 2.17000e+00 3.22920e+00
 2.08000e-02 9.40000e-02 1.47000e-02 4.74000e-02 3.70000e-03 2.97450e+00
 7.33424e+01 7.28090e+01 7.29326e+01 7.33773e+01]
6
[[3.32970e+00 1.41700e-01 6.41900e-01 1.00200e-01 3.33600e-01 4.68000e-02
  2.78340e+00 3.01860e+00 4.21200e-01 1.73810e+00 2.97800e-01 8.99000e-01
  2.95000e-01 2.60680e+00 2.48500e+00 6.97700e-01 2.36980e+00 4.93400e-01
  1.22600e+00 4.92400e-01 2.29860e+00 3.48270e+00 2.39300e-01 1.17100e+00
  1.69200e-01 5.89300e-01 1.08500e-01 2.94480e+00 2.48490e+00 4.77200e-01
  1.64680e+00 3.37400e-01 8.38500e-01 3.34900e-01 2.17000e+00 3.

# TIME SERIES

In [21]:

from sklearn.model_selection import TimeSeriesSplit


In [22]:
X_train['wf_start_time'] =  pd.to_datetime(X_train['wf_start_time'], format='%Y-%m-%d %H:%M:%S.%f')


In [23]:
from datetime import datetime
min_date = X_train.wf_start_time.min()
max_date = X_train.wf_start_time.max()


print("Min:", min_date, "Max:", max_date)


Min: 2022-08-09 14:03:22.000007 Max: 2022-09-01 20:11:32.000007


In [24]:
from datetime import datetime as dt




train_percent = .75

time_between = max_date - min_date
train_cutoff = min_date + train_percent*time_between
train_cutoff



Timestamp('2022-08-27 00:39:29.500007')

In [25]:
pd.DataFrame(X_train.index==y_train.index).value_counts()


True    179041
dtype: int64

In [26]:
#JOIN AGAIN X_TRAIN AND Y_TRAIN
# df.merge(s.to_frame(), left_index=True, right_index=True)
X_train=X_train.merge(y_train.to_frame(name='rul'), left_index=True, right_index=True)



In [27]:
#HERE I CREATED AN INNER SPLIT THAT CAN BE USED AS VALIDATION!
_X_train = X_train[X_train.wf_start_time <= train_cutoff]
_X_test = X_train[X_train.wf_start_time > train_cutoff]
print("Train:", _X_train.status.unique())
print("Test:", _X_test.status.unique())
print("Train:", _X_train.wf_start_time.min(), _X_train.wf_start_time.max())
print("Test:", _X_test.wf_start_time.min(), _X_test.wf_start_time.max())


Train: ['1mm_Offset' 'No_Offset' '3mm_Offset' '6.25mm_Offset']
Test: ['6.25mm_Offset' '12.7mm_Offset']
Train: 2022-08-09 14:03:22.000007 2022-08-22 18:46:14.000007
Test: 2022-08-30 19:17:41.000007 2022-09-01 20:11:32.000007


In [28]:

_X_train = X_train[X_train.columns.drop(list(X_train.filter(regex='status')))] # drop status column


In [29]:

_X_train=_X_train.sort_values('wf_start_time', ignore_index = True)


target_cols = ['rul']
non_target_cols = list(set(_X_train.columns) - set(target_cols + ['wf_start_time']))



In [30]:
print(target_cols)
print(non_target_cols)



['rul']
['Probe_3_RadialHorizontal_Peak~Peak_g', 'Probe_4_RadialVertical_TruePeak_g', 'Probe_5_RadialHorizontal_TruePeak_g', 'Probe_1_AxialDirection_Kurtosis_g~g', 'Probe_4_RadialVertical_RMS_g', 'Probe_2_RadialVertical_Kurtosis_g~g', 'Thermocouple 2_Value', 'Probe_5_RadialHorizontal_RMS_g', 'Probe_6_BearingRadial_CrestFactor_g~g', 'Probe_4_RadialVertical_Kurtosis_g~g', 'Probe_5_RadialHorizontal_Kurtosis_g~g', 'Thermocouple 3_Value', 'Probe_3_RadialHorizontal_TruePeak_g', 'Probe_2_RadialVertical_HighFrequency_grms', 'Probe_2_RadialVertical_RMS_g', 'Probe_1_AxialDirection_CrestFactor_g~g', 'Probe_5_RadialHorizontal_CrestFactor_g~g', 'Probe_5_RadialHorizontal_Peak~Peak_g', 'Probe_2_RadialVertical_DerivedPeak_g', 'Probe_4_RadialVertical_DerivedPeak_g', 'Probe_4_RadialVertical_Peak~Peak_g', 'Probe_6_BearingRadial_RMS_g', 'Probe_2_RadialVertical_Peak~Peak_g', 'Probe_4_RadialVertical_CrestFactor_g~g', 'Probe_1_AxialDirection_Peak~Peak_g', 'Probe_5_RadialHorizontal_HighFrequency_grms', 'Probe

In [31]:
X = _X_train[non_target_cols]
y = _X_train[target_cols]

In [32]:
scaler = MinMaxScaler()

X=X.values
y=y.values

In [33]:
len(X_train)

179041

In [34]:
len(y_train)

179041

In [35]:
import tensorflow as tf

In [36]:
#!mkdir logs/fit/
# !rm -rf logs/fit/*
# !sudo firewall-cmd  --add-port=8000/tcp

In [37]:
# %load_ext tensorboard
# %tensorboard --logdir=/tmp/fit  --port=8001 --host=0.0.0.0

In [38]:
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

lr = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 10**-7 * 10**(epoch/3))

es=EarlyStopping(
    monitor="val_loss",

    patience=3,
    verbose=1,
    mode="auto",
)

# # checkpoint_filepath = "./tmp/checkpoint"
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import L1L2
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

NameError: name 'sys' is not defined

In [39]:
tscv = TimeSeriesSplit(max_train_size=None, n_splits=2)
print(tscv)
tf.keras.backend.clear_session()

for train_index, val_index in tscv.split(X):
    
    
    print("TRAIN:", train_index, "TEST:", val_index) 
    _X_train, _X_val = X[train_index], X[val_index]
    print(_X_train)
    
    _y_train, _y_val = y[train_index], y[val_index]

    scaler = MinMaxScaler()
    _X_val=scaler.fit_transform(_X_val)
    
    #TRAINING
    out_seq = np.array(_y_train)
    # horizontally stack columns
    dataset = np.array(_X_train)

    # shift the target sample by one step
    out_seq = np.insert(out_seq, 0, 0)
    #pre
    out_seq = np.delete(out_seq, -1)
    # define generator
    n_input = SEQUENCE_LENGHT #leght of the sequence timesteps
    
    training_generator = TimeseriesGenerator(dataset, out_seq, length=n_input, batch_size=STEPS_PER_BATCH)
#     for i in range(len(training_generator)):
#         a, b = training_generator[i]
#         print(a.shape, b.shape)


    # define generator
    n_features = 46

    # define model
# model = Sequential()

# model.add(LSTM(32, activation='relu',return_sequences=False, input_shape=(n_input, n_features), kernel_regularizer = L1L2(l1=0.01, l2=0.0)))
# #     model.add(Dropout(0.4))
# #     model.add(LSTM(20,return_sequences=False))
# #     model.add(Dropout(0.5))
# #     model.add(tf.keras.layers.Dense(4, activation='relu'))
# model.add(Dense(1))
# model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001), loss='mse')#optimizer=tf.keras.optimizers.RMSprop(

    model = tf.keras.models.Sequential([
        # "1" 
        tf.keras.layers.LSTM(64, input_shape = (n_input, n_features), return_sequences = True),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dropout(0.2),
        #"2"
        tf.keras.layers.Dense(1, activation = 'relu')
    ])

    model.compile(loss=tf.keras.losses.Huber(), optimizer = tf.keras.optimizers.Adam(lr = 10**-1), metrics =['mse','mae'])


    scaler = MinMaxScaler()
    _X_val=scaler.fit_transform(_X_val)

    #VALIDATION
    out_seq = np.array(_y_val)
    # horizontally stack columns
    dataset = np.array(_X_val)

    # shift the target sample by one step
    out_seq = np.insert(out_seq, 0, 0)
    #pre
    out_seq = np.delete(out_seq, -1)
    # define generator
    n_input = SEQUENCE_LENGHT #leght of the sequence timesteps
    STEPS_PER_BATCH=32
    validation_generator = TimeseriesGenerator(dataset, out_seq, length=n_input, batch_size=STEPS_PER_BATCH)
#     for i in range(len(validation_generator)):
#         x, y = validation_generator[i]
#         print('%s => %s' % (x, y))
    
    
    history = model.fit(training_generator,
                                            steps_per_epoch=STEPS_PER_BATCH,
                                            epochs=400,
                                            verbose=1,
                                              callbacks=[es,tensorboard_callback],
                                            validation_data=validation_generator) # class_weight=class_weights)

#HIGHER THE SEQUENCE LOSS GOES TO INFINITY

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None)
TRAIN: [    0     1     2 ... 59678 59679 59680] TEST: [ 59681  59682  59683 ... 119358 119359 119360]
[[ 8.3688  6.7436  5.9486 ...  0.3655  0.9138  2.5585]
 [20.0644  7.3079  7.0093 ...  1.8019  0.5488  3.8151]
 [14.366   6.5833  6.1806 ...  1.802   0.5146  4.8633]
 ...
 [ 2.6698  0.6054  0.8314 ...  0.566   0.3386  0.9543]
 [ 2.6498  0.6476  0.7592 ...  0.564   0.3802  0.861 ]
 [ 2.5786  0.6167  0.8086 ...  0.5625  0.3652  0.9296]]


2023-05-03 23:31:18.983299: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 188 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0
2023-05-03 23:31:19.013327: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:736] failed to allocate 188.69MiB (197853184 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-05-03 23:31:19.212965: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-03 23:31:19.214099: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate

Epoch 1/400


2023-05-03 23:31:19.516032: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-03 23:31:19.739994: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-03 23:31:19.741466: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

UnknownError: Graph execution error:

Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/lstm/PartitionedCall]] [Op:__inference_train_function_6018]

In [31]:
## utility - save/load the model
import os
import joblib

# to save
# joblib.dump(rf, path + '/results/random_forest_offset_RUL.joblib') 

# to load
rf = joblib.load(data_path + '/random_forest_offset_RUL.joblib')

FileNotFoundError: [Errno 2] No such file or directory: '/home/jose/FL_AM_Defect-Detection/fl_testbed/version2/data/initial/random_forest_offset_RUL.joblib'

### Evaluation

In [ ]:
y_pred = rf.predict(X_test) ## using the untinted dataset!
    
print('R^2:', metrics.r2_score(y_test, y_pred))
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))
print('Mean Absolute Percentage Error (MAPE):', metrics.mean_absolute_percentage_error(y_test, y_pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) # np.sqrt

print('Explained Variance Score:', metrics.explained_variance_score(y_test, y_pred))
print('Max Error:', metrics.max_error(y_test, y_pred))
print('Mean Squared Log Error:', metrics.mean_squared_log_error(y_test, y_pred))
print('Median Absolute Error:', metrics.median_absolute_error(y_test, y_pred))

## with n_estimators = 150

# R^2: 0.9991654290573937 1882843
# Mean Absolute Error (MAE): 1158.6216816072117
# Mean Squared Error (MSE): 333818221.94037396
# Root Mean Squared Error (RMSE): 18270.692979205083
# Explained Variance Score: 0.9991654316522736
# Max Error: 1998236.0866666667
# Mean Squared Log Error: 0.012717975468372306
# Median Absolute Error: 147.4266666667536

## n_estimators = 10
# R^2: 0.9990822069686238
# Mean Absolute Error (MAE): 1213.5715068447246
# Mean Squared Error (MSE): 367106044.79767376
# Mean Absolute Percentage Error (MAPE): 1.0690696275079867e+17
# Root Mean Squared Error (RMSE): 19160.011607451437
# Explained Variance Score: 0.9990822141398445
# Max Error: 1998818.5
# Mean Squared Log Error: 0.012296657672986336
# Median Absolute Error: 137.60000000009313
1262373.8750
1381205.3750

### Show feature importance

In [32]:
# # RF: Get feature list
# ## Learn more: https://towardsdatascience.com/explaining-feature-importance-by-example-of-a-random-forest-d9166011959e

# features = list(X_test.columns)  

# rf_imp_features = []

# ## Plot the feature importance
# def plot_feature_importance ():
#     importances = rf.feature_importances_
    
#     indices = np.argsort(importances)[len(importances)-25:] ## top 25    

#     plt.figure(figsize = (12,8))
#     plt.rcParams['font.size'] = 12
    
#     plt.title('Feature Importances')
#     plt.barh(range(len(indices)), importances[indices], color='b', align='center')

#     features_y = []
#     for x in indices:
#         features_y.append(features[x])
        
#     plt.yticks(range(len(indices)), features_y) 

#     plt.xlabel('Relative Importance')
#     plt.savefig(path + '/results/fi_offset_RUL.png') # save

#     plt.show()
#     return features_y

# rf_imp_features = plot_feature_importance()

### Prediction

In [ ]:
# get a random sample to verify the results!
## 
import matplotlib
import matplotlib.pyplot as plt
RNDSEED = np.random.seed(39)

dfs = df.sample(1000, random_state = RNDSEED) # data points
# print(dfs.rul)
X_dfs = dfs.drop('rul',axis = 1) # drop response

ys_pred = rf.predict(X_dfs)
# print(ys_pred)

rul = dfs[['rul']]
rul = rul.rename(columns={'rul': 'original'})
rul['prediction'] = ys_pred
# rul.head(5)

matplotlib.style.use('ggplot') ## styling

rul.plot.scatter(x='original', y='prediction', figsize=(16, 9), c='original', colormap='viridis') ## scatter

plt.savefig(path + '/results/offset_rul_prediction.png') # save


In [ ]:
## Plot the % of deviation 

rul['difference'] = (rul['prediction'] - rul['original']) /  rul['original']

rul.index = range(len(rul.index)) ## reset index

ax = rul.plot( y=["difference"], figsize=(16, 9))

ax.set_xlabel("Index")
ax.set_ylabel("Difference")

plt.show()

# out of couriosity 
print(len(rul[(rul['difference'] > 0.2) | (rul['difference'] < -0.2)])) # how many more than 20%? # 8 only!

In [ ]:
## styles
plt.style.available


# Working LSTM

In [291]:
# multivariate one step problem
from numpy import array
from numpy import hstack
from numpy import insert
from keras.preprocessing.sequence import TimeseriesGenerator
# define dataset
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])
out_seq = array([25, 45, 65, 85, 105, 125, 145, 165, 185, 205])
# reshape series

In [292]:

in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2))

In [293]:
pd.DataFrame(dataset)



,0,1
0,10,15
1,20,25
2,30,35
3,40,45
4,50,55
5,60,65
6,70,75
7,80,85
8,90,95
9,100,105


In [294]:
#NOT DIRECTLY GO BACK ONE TIMESTEP !!!!!NOT!!!! BAD WAY
generator = TimeseriesGenerator(dataset, out_seq, length=n_input, batch_size=1)

for i in range(len(generator)):
    x, y = generator[i]
    print('%s => %s' % (x, y))

[[[10 15]]] => [[45]]
[[[20 25]]] => [[65]]
[[[30 35]]] => [[85]]
[[[40 45]]] => [[105]]
[[[50 55]]] => [[125]]
[[[60 65]]] => [[145]]
[[[70 75]]] => [[165]]
[[[80 85]]] => [[185]]
[[[90 95]]] => [[205]]


In [295]:

# shift the target sample by one step
out_seq = insert(out_seq, 0, 0)
#pre

out_seq = np.delete(out_seq, -1)
# define generator
n_input = 1
generator = TimeseriesGenerator(dataset, out_seq, length=n_input, batch_size=1)
# print each sample
for i in range(len(generator)):
    x, y = generator[i]
    print('%s => %s' % (x, y))

[[[10 15]]] => [25]
[[[20 25]]] => [45]
[[[30 35]]] => [65]
[[[40 45]]] => [85]
[[[50 55]]] => [105]
[[[60 65]]] => [125]
[[[70 75]]] => [145]
[[[80 85]]] => [165]
[[[90 95]]] => [185]


In [296]:
# define generator
n_features = dataset.shape[1]
n_input = SEQUENCE_LENGHT
generator = TimeseriesGenerator(dataset, dataset, length=n_input, batch_size=8)
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit_generator(generator, steps_per_epoch=1, epochs=500, verbose=1)


Epoch 1/500


/tmp/ipykernel_413302/1060287710.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator, steps_per_epoch=1, epochs=500, verbose=1)
2023-05-03 13:28:14.868586: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1/1 [==============================] - 1s 1s/step - loss: 4717.5898
Epoch 2/500
1/1 [==============================] - 0s 16ms/step - loss: 4642.8096
Epoch 3/500
1/1 [==============================] - 0s 15ms/step - loss: 4569.4141
Epoch 4/500
1/1 [==============================] - 0s 15ms/step - loss: 4497.0552
Epoch 5/500
1/1 [==============================] - 0s 15ms/step - loss: 4424.9897
Epoch 6/500
1/1 [==============================] - 0s 15ms/step - loss: 4353.1387
Epoch 7/500
1/1 [==============================] - 0s 16ms/step - loss: 4280.8086
Epoch 8/500
1/1 [==============================] - 0s 14ms/step - loss: 4207.4229
Epoch 9/500
1/1 [==============================] - 0s 15ms/step - loss: 4132.2603
Epoch 10/500
1/1 [==============================] - 0s 14ms/step - loss: 4055.1626
Epoch 11/500
1/1 [==============================] - 0s 15ms/step - loss: 3975.5200
Epoch 12/500
1/1 [==============================] - 0s 14ms/step - loss: 3892.7153
Epoch 13/500
1/1 [=========

1/1 [==============================] - 0s 16ms/step - loss: 6.3450
Epoch 203/500
1/1 [==============================] - 0s 15ms/step - loss: 6.3416
Epoch 204/500
1/1 [==============================] - 0s 15ms/step - loss: 6.3385
Epoch 205/500
1/1 [==============================] - 0s 15ms/step - loss: 6.3355
Epoch 206/500
1/1 [==============================] - 0s 14ms/step - loss: 6.3328
Epoch 207/500
1/1 [==============================] - 0s 15ms/step - loss: 6.3303
Epoch 208/500
1/1 [==============================] - 0s 15ms/step - loss: 6.3278
Epoch 209/500
1/1 [==============================] - 0s 17ms/step - loss: 6.3252
Epoch 210/500
1/1 [==============================] - 0s 15ms/step - loss: 6.3225
Epoch 211/500
1/1 [==============================] - 0s 14ms/step - loss: 6.3196
Epoch 212/500
1/1 [==============================] - 0s 15ms/step - loss: 6.3171
Epoch 213/500
1/1 [==============================] - 0s 14ms/step - loss: 6.3149
Epoch 214/500
1/1 [=======================

1/1 [==============================] - 0s 15ms/step - loss: 6.2548
Epoch 405/500
1/1 [==============================] - 0s 15ms/step - loss: 6.2547
Epoch 406/500
1/1 [==============================] - 0s 14ms/step - loss: 6.2547
Epoch 407/500
1/1 [==============================] - 0s 14ms/step - loss: 6.2546
Epoch 408/500
1/1 [==============================] - 0s 14ms/step - loss: 6.2546
Epoch 409/500
1/1 [==============================] - 0s 15ms/step - loss: 6.2546
Epoch 410/500
1/1 [==============================] - 0s 15ms/step - loss: 6.2546
Epoch 411/500
1/1 [==============================] - 0s 15ms/step - loss: 6.2546
Epoch 412/500
1/1 [==============================] - 0s 15ms/step - loss: 6.2546
Epoch 413/500
1/1 [==============================] - 0s 15ms/step - loss: 6.2546
Epoch 414/500
1/1 [==============================] - 0s 15ms/step - loss: 6.2545
Epoch 415/500
1/1 [==============================] - 0s 15ms/step - loss: 6.2545
Epoch 416/500
1/1 [=======================

In [298]:
# make a one step prediction out of sample
x_input = array([[90, 95], [100, 105]]).reshape((1, n_input, n_features))
yhat = model.predict(x_input, verbose=1)
print(yhat)

1/1 [==============================] - 0s 18ms/step
[[112.921326]]
